In [30]:
# from datetime import datetime
import locale
from model import edit_type
from model import book

import re
pattern_1 = re.compile(u' Added on [A-Za-z]+, \d{1,2}')
pattern_2 = re.compile(u' Added on [A-Za-z]+, [A-Za-z]+')
pattern_3 = re.compile(u' 添加于 \d{4}')
def parse_edit_date(data):
    date_part = data.split("|")
    added_time = ""
    if(pattern_1.search(date_part[-1])):
        locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
        added_time = datetime.strptime(date_part[-1]," Added on %A, %d %B %Y %H:%M:%S")
    elif(pattern_2.search(date_part[-1])): 
        locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
        added_time = datetime.strptime(date_part[-1]," Added on %A, %B %d, %Y %I:%M:%S %p")
    elif(pattern_3.search(date_part[-1])):
        locale.setlocale(locale.LC_ALL, 'zh_CN.UTF-8')
        added_time = datetime.strptime(date_part[-1]," 添加于 %Y年%m月%d日%A %p%H:%M:%S")
    else:
        print("{} can't be parsed".format(date_part[-1]))
    
    return added_time


def parse_edit_location(data):                                                                                  
    """                                                                                                               
          Example of input:                                                                                                
         - Your Note on Location 4555 | Added on Wednesday, February 24, 2016 8:28:14 AM                                                                                                                                                            
          :param data:                                                                                                      
          :return:                                                                                                         
          """                                                                                                              
    loc_split = data.split("|")                                                                                      
    if len(loc_split) > 2:                                                                                           
        return loc_split[1].rstrip().lstrip().split(" ")[-1]                                                          
    else:                                                                                                            
  #kahar add                                                                                                    
        if("您" in loc_split[0]):                                                                                     
            print("loc_split:",loc_split[0])                                                                          
            return loc_split[0].split(" ")[2].strip("#")                                                              
        else:                                                                                                       
            print("en loc:",loc_split[0])                                                                            
            return loc_split[0].split(" ")[5]

In [31]:
def parse_raw(filename,book):

    with open(filename, 'r', encoding='utf-8') as file:
        # Read content into one string
        data_raw = file.read()

    # This will split the string by edits (Look at the raw format of the file)
    # Last string is deleted  because there is no content after the last
    # ====== in the file.
    lines = data_raw.split("==========")[0:-1]
    # Append new line so all lines have \n at the beginning
    lines[0] = "\n" + lines[0]
    #kahar add
#     print("parse_raw lines:",lines)
    books_created = {}
    
    for edit in lines:
        # Split edit by new line
        print("edit:",edit)
        header, book_name, meta, blank, content, footer1 = edit.split('\n')
#         print("header={},book_name={},meta={},blank={},content={},footer1={}"\
#              .format(header,book_name,meta,blank,content,footer1))
        # Remove Byte Order Mark (BOM) code
        book_name = book_name.replace(u'\ufeff', '')
        # Check if this type of books exists
        if book_name not in books_created:
            books_created[book_name] = Book()
            books_created[book_name].book_name = book_name

        create_edit(meta=meta, content=content,
                         book=books_created[book_name])
    return books_created 

def create_edit(meta, content, book):
    if("您" in meta):#中文
#         print("zhongwen")
        if  "标注" in meta:
            print("meta='{}'".format(meta))
            print("content='{}'".format(content))
            edit = edit_type.HighlightType(highlight_string=meta,
                                       content=content)
            book.highlights_list.append(edit)
        
        elif "笔记" in meta:
        # It's a note type
            edit = edit_type.NoteType(edit_string=meta, content=content)
            book.notes_list.append(edit)
        
        elif "书签" in meta:
            edit = edit_type.BookmarkType(bookmark_string=meta)
            book.bookmarks_list.append(edit)
        else:
            print("there is exception in Chinese meta=",meta)

    else:
        type_of_edit = meta.split(" ")[2]
#         print("type_of_edit",type_of_edit)

        if  type_of_edit.lower() == "bookmark":
            edit = edit_type.BookmarkType(bookmark_string=meta)
            book.bookmarks_list.append(edit)

        elif type_of_edit.lower() == "highlight":
            edit = edit_type.HighlightType(highlight_string=meta,
                                           content=content)
            book.highlights_list.append(edit)

        elif type_of_edit.lower() == "note":
            # It's a note type
            edit = edit_type.NoteType(edit_string=meta, content=content)
            book.notes_list.append(edit)
        else:
            print("there is exception in English meta={},type_of_edit={}"\
                  ,meta,type_of_edit)
    
    
        

In [32]:
        
filename = "/home/kaharjan/gitprojects/for_kindle/My Clippings.txt"         
        
data = "186  - Your Highlight on pa添加于 2018年3月26日星期一 下午7:23:46ge 42 | Added on Wednesday, October 21, 2015 10:45:24 PM"
data = " Added on Monday, 23 November 2015 23:32:48"
data = " Added on Monday, 23 November 2015 23:32:48"
data =" Added on Thursday, 7 December 2017 00:04:57"
data ="- 您在位置 #98 的笔记 | 添加于 2018年3月26日星期一 下午7:23:46"
#data = "- Your Note on Location 4555 | Added on Wednesday, February 24, 2016 8:28:14 AM  "
# data = " 添加于 2018年3月26日星期一 下午7:23:46"
# data = " 添加于 2018年3月26日星期一 下午7:23:46"
# data = "Added on Wednesday, October 21, 2015 10:45:24 PM"
# data = " Added on Wednesday, October 21, 2015 10:45:24 PM"
# data ='\nReminiscences of a Stock Operator (Edwin Lefe     vre)\n- 您在位置 #72-72的标注 | 添加于 2022年4月13日星期三 上午7:40:46\n\ndickens\n'
# data = '\nReminiscences of a Stock Operator (Edwin Lefevre)\n- 您在位置 #80-80的标注 | 添加于 2022年4月13日星期三 上午7:43:22\n\ndandy\n'
data = 'Reminiscences of a Stock Operator (Edwin Lefevre)- 您在位置 #80-80的标注 | 添加于 2022年4月13日星期三 上午7:43:22'
data = '- 您在位置 #80-80的标注 | 添加于 2022年4月13日星期三 上午7:43:22'
# print(parse_edit_date(data))
print(parse_edit_location(data))
book = Book()
books = parse_raw(filename,books)

loc_split: - 您在位置 #80-80的标注 
80-80的标注
edit: 
﻿186  
- Your Bookmark on page 25 | Added on Sunday, October 18, 2015 8:05:13 PM



en loc: - Your Bookmark on page 25 
edit: 
186  
- Your Note on page 27 | Added on Sunday, October 18, 2015 11:24:55 PM

heyday 

en loc: - Your Note on page 27 
edit: 
186  
- Your Highlight on page 27 | Added on Sunday, October 18, 2015 11:24:55 PM

heyday

en loc: - Your Highlight on page 27 
edit: 
186  
- Your Bookmark on page 30 | Added on Sunday, October 18, 2015 11:47:55 PM



en loc: - Your Bookmark on page 30 
edit: 
186  
- Your Highlight on page 31 | Added on Monday, October 19, 2015 11:43:53 PM

verse

en loc: - Your Highlight on page 31 
edit: 
186  
- Your Highlight on page 32 | Added on Monday, October 19, 2015 11:48:49 PM

Semitic

en loc: - Your Highlight on page 32 
edit: 
186  
- Your Highlight on page 33 | Added on Monday, October 19, 2015 11:49:32 PM

protagonists

en loc: - Your Highlight on page 33 
edit: 
186  
- Your Highlight on pag

loc_split: - 您在位置 #186 的笔记 
edit: 
Pride and Prejudice (Jane Austen)
- Your Highlight on Location 201-201 | Added on Tuesday, March 27, 2018 3:12:14 PM

sprained

en loc: - Your Highlight on Location 201-201 
edit: 
Pride and Prejudice (Jane Austen)
- Your Highlight on Location 211-211 | Added on Tuesday, March 27, 2018 3:14:39 PM

breeding!

en loc: - Your Highlight on Location 211-211 
edit: 
Pride and Prejudice (Jane Austen)
- Your Highlight on Location 215-215 | Added on Tuesday, March 27, 2018 3:16:46 PM

gallantry

en loc: - Your Highlight on Location 215-215 
edit: 
Pride and Prejudice (Jane Austen)
- Your Highlight on Location 220-220 | Added on Tuesday, March 27, 2018 3:34:54 PM

candour

en loc: - Your Highlight on Location 220-220 
edit: 
Pride and Prejudice (Jane Austen)
- Your Highlight on Location 220-221 | Added on Tuesday, March 27, 2018 3:35:08 PM

candid

en loc: - Your Highlight on Location 220-221 
edit: 
Pride and Prejudice (Jane Austen)
- Your Highlight on Locatio

content='釜'
loc_split: - 您在位置 #1307-1307的标注 
edit: 
人类简史：从动物到上帝 (尤瓦尔·赫拉利)
- 您在位置 #1322-1322的标注 | 添加于 2018年4月6日星期五 下午10:25:51

斟

meta='- 您在位置 #1322-1322的标注 | 添加于 2018年4月6日星期五 下午10:25:51'
content='斟'
loc_split: - 您在位置 #1322-1322的标注 
edit: 
人类简史：从动物到上帝 (尤瓦尔·赫拉利)
- 您在位置 #1322-1322的标注 | 添加于 2018年4月6日星期五 下午10:25:55

酌

meta='- 您在位置 #1322-1322的标注 | 添加于 2018年4月6日星期五 下午10:25:55'
content='酌'
loc_split: - 您在位置 #1322-1322的标注 
edit: 
人类简史：从动物到上帝 (尤瓦尔·赫拉利)
- 您在位置 #1325-1325的标注 | 添加于 2018年4月7日星期六 上午10:46:03

躁

meta='- 您在位置 #1325-1325的标注 | 添加于 2018年4月7日星期六 上午10:46:03'
content='躁'
loc_split: - 您在位置 #1325-1325的标注 
edit: 
人类简史：从动物到上帝 (尤瓦尔·赫拉利)
- 您在位置 #1333-1333的标注 | 添加于 2018年4月7日星期六 上午10:48:27

谷

meta='- 您在位置 #1333-1333的标注 | 添加于 2018年4月7日星期六 上午10:48:27'
content='谷'
loc_split: - 您在位置 #1333-1333的标注 
edit: 
人类简史：从动物到上帝 (尤瓦尔·赫拉利)
- 您在位置 #1338-1338的标注 | 添加于 2018年4月7日星期六 上午10:49:57

咎

meta='- 您在位置 #1338-1338的标注 | 添加于 2018年4月7日星期六 上午10:49:57'
content='咎'
loc_split: - 您在位置 #1338-1338的标注 
edit: 
人类简史：从动物到上帝 (

loc_split: - 您在位置 #4184-4184的标注 
edit: 
人类简史：从动物到上帝 (尤瓦尔·赫拉利)
- 您在位置 #4194-4194的标注 | 添加于 2018年4月27日星期五 下午3:35:22

碑

meta='- 您在位置 #4194-4194的标注 | 添加于 2018年4月27日星期五 下午3:35:22'
content='碑'
loc_split: - 您在位置 #4194-4194的标注 
edit: 
人类简史：从动物到上帝 (尤瓦尔·赫拉利)
- 您在位置 #4195-4195的标注 | 添加于 2018年4月27日星期五 下午3:35:43

甚

meta='- 您在位置 #4195-4195的标注 | 添加于 2018年4月27日星期五 下午3:35:43'
content='甚'
loc_split: - 您在位置 #4195-4195的标注 
edit: 
人类简史：从动物到上帝 (尤瓦尔·赫拉利)
- 您在位置 #4195-4195的标注 | 添加于 2018年4月27日星期五 下午3:35:58

知之甚详

meta='- 您在位置 #4195-4195的标注 | 添加于 2018年4月27日星期五 下午3:35:58'
content='知之甚详'
loc_split: - 您在位置 #4195-4195的标注 
edit: 
人类简史：从动物到上帝 (尤瓦尔·赫拉利)
- 您在位置 #4196-4196的标注 | 添加于 2018年4月27日星期五 下午3:36:28

敞

meta='- 您在位置 #4196-4196的标注 | 添加于 2018年4月27日星期五 下午3:36:28'
content='敞'
loc_split: - 您在位置 #4196-4196的标注 
edit: 
人类简史：从动物到上帝 (尤瓦尔·赫拉利)
- 您在位置 #4196-4196的标注 | 添加于 2018年4月27日星期五 下午3:49:03

喧

meta='- 您在位置 #4196-4196的标注 | 添加于 2018年4月27日星期五 下午3:49:03'
content='喧'
loc_split: - 您在位置 #4196-4196的标注 
edit: 
人类简史：从动物到上帝 (尤瓦尔·赫拉

meta='- 您在位置 #5124-5124的标注 | 添加于 2018年5月3日星期四 下午6:21:55'
content='购'
loc_split: - 您在位置 #5124-5124的标注 
edit: 
人类简史：从动物到上帝 (尤瓦尔·赫拉利)
- 您在位置 #5124-5124的标注 | 添加于 2018年5月3日星期四 下午6:22:05

衬

meta='- 您在位置 #5124-5124的标注 | 添加于 2018年5月3日星期四 下午6:22:05'
content='衬'
loc_split: - 您在位置 #5124-5124的标注 
edit: 
人类简史：从动物到上帝 (尤瓦尔·赫拉利)
- 您在位置 #5124-5124的标注 | 添加于 2018年5月3日星期四 下午6:22:09

衬衫和

meta='- 您在位置 #5124-5124的标注 | 添加于 2018年5月3日星期四 下午6:22:09'
content='衬衫和'
loc_split: - 您在位置 #5124-5124的标注 
edit: 
人类简史：从动物到上帝 (尤瓦尔·赫拉利)
- 您在位置 #5125-5125的标注 | 添加于 2018年5月3日星期四 下午6:22:22

粉

meta='- 您在位置 #5125-5125的标注 | 添加于 2018年5月3日星期四 下午6:22:22'
content='粉'
loc_split: - 您在位置 #5125-5125的标注 
edit: 
人类简史：从动物到上帝 (尤瓦尔·赫拉利)
- 您在位置 #5125-5125的标注 | 添加于 2018年5月3日星期四 下午6:22:31

素

meta='- 您在位置 #5125-5125的标注 | 添加于 2018年5月3日星期四 下午6:22:31'
content='素'
loc_split: - 您在位置 #5125-5125的标注 
edit: 
人类简史：从动物到上帝 (尤瓦尔·赫拉利)
- 您在位置 #5128-5128的标注 | 添加于 2018年5月3日星期四 下午6:23:18

磐

meta='- 您在位置 #5128-5128的标注 | 添加于 2018年5月3日星期四 下午6:23:18'
content='磐'
lo

content='蜥'
loc_split: - 您在位置 #1107-1107的标注 
edit: 
未来简史 ([以色列]尤瓦尔·赫拉利)
- 您在位置 #1117-1117的标注 | 添加于 2018年6月30日星期六 下午1:01:47

畜

meta='- 您在位置 #1117-1117的标注 | 添加于 2018年6月30日星期六 下午1:01:47'
content='畜'
loc_split: - 您在位置 #1117-1117的标注 
edit: 
未来简史 ([以色列]尤瓦尔·赫拉利)
- 您在位置 #1125-1125的标注 | 添加于 2018年6月30日星期六 下午1:04:32

鳄

meta='- 您在位置 #1125-1125的标注 | 添加于 2018年6月30日星期六 下午1:04:32'
content='鳄'
loc_split: - 您在位置 #1125-1125的标注 
edit: 
未来简史 ([以色列]尤瓦尔·赫拉利)
- 您在位置 #1133-1134的标注 | 添加于 2018年6月30日星期六 下午1:11:14

进化论认为，所有本能、冲动、情感的进化都只有一个目的：生存和繁衍。

meta='- 您在位置 #1133-1134的标注 | 添加于 2018年6月30日星期六 下午1:11:14'
content='进化论认为，所有本能、冲动、情感的进化都只有一个目的：生存和繁衍。'
loc_split: - 您在位置 #1133-1134的标注 
edit: 
未来简史 ([以色列]尤瓦尔·赫拉利)
- 您在位置 #1146-1146的标注 | 添加于 2018年6月30日星期六 下午1:15:36

芳

meta='- 您在位置 #1146-1146的标注 | 添加于 2018年6月30日星期六 下午1:15:36'
content='芳'
loc_split: - 您在位置 #1146-1146的标注 
edit: 
未来简史 ([以色列]尤瓦尔·赫拉利)
- 您在位置 #1149-1149的标注 | 添加于 2018年6月30日星期六 下午1:16:02

野

meta='- 您在位置 #1149-1149的标注 | 添加于 2018年6月30日星期六 下午1:16:02'
content='野'

edit: 
价值(高瓴创始人兼首席执行官张磊首部力作;沉淀15年，张磊的投资思想首度全面公开) (张磊)
- 您在位置 #422-424的标注 | 添加于 2021年7月4日星期日 下午2:34:13

经济学家约翰·梅纳德·凯恩斯（John Maynard Keynes）有句名言：“处世的智慧教导人们宁可依循传统而失败，也不愿意打破传统而成功。”（Worldly Wisdom teaches that it is better for reputation to fail conventionally than to succeed unconventionally.）

meta='- 您在位置 #422-424的标注 | 添加于 2021年7月4日星期日 下午2:34:13'
content='经济学家约翰·梅纳德·凯恩斯（John Maynard Keynes）有句名言：“处世的智慧教导人们宁可依循传统而失败，也不愿意打破传统而成功。”（Worldly Wisdom teaches that it is better for reputation to fail conventionally than to succeed unconventionally.）'
loc_split: - 您在位置 #422-424的标注 
edit: 
价值(高瓴创始人兼首席执行官张磊首部力作;沉淀15年，张磊的投资思想首度全面公开) (张磊)
- 您在位置 #425 的笔记 | 添加于 2021年7月4日星期日 下午2:34:40

成语

loc_split: - 您在位置 #425 的笔记 
edit: 
价值(高瓴创始人兼首席执行官张磊首部力作;沉淀15年，张磊的投资思想首度全面公开) (张磊)
- 您在位置 #424-425的标注 | 添加于 2021年7月4日星期日 下午2:34:40

循规蹈矩

meta='- 您在位置 #424-425的标注 | 添加于 2021年7月4日星期日 下午2:34:40'
content='循规蹈矩'
loc_split: - 您在位置 #424-425的标注 
edit: 
价值(高瓴创始人兼首席执行官张磊首部力作;沉淀15年，张磊的投资思想首度全面公开) (张磊)
- 您在位置 #422-427的标注 | 添加于 202

In [55]:
b=""
for i,b in enumerate(books):
    print(i,":book name:",b)
    print("highlights_count:",books[b].highlights_count())
    print("bookmarks_count:",books[b].bookmarks_count())
#     print("bookmarks_list:",books[b].bookmarks_list[0][0])
    print("booknote_count:",books[b].notes_count())
#     break
#     print(books[b].start_finish_reading_date())
    print(books[b].end_reading_date)
#     print(book.bookmarks)
#     print("       bookmarks")
#     books[book].bookmarks_count:
#         print(bookmark)
#     print(books[book].bookmarks_count)
#     print(books[book].highlights_count)
#     print(books[book].notes_count)
# books.bookmarks

0 :book name: 186  
highlights_count: 288
bookmarks_count: 52
booknote_count: 4
2023-05-13
1 :book name: GrammarAsScience  
highlights_count: 16
bookmarks_count: 6
booknote_count: 0
2023-05-13
2 :book name: Moon and Sixpence (免费公版书) (Maugham,W. Somerset (William Somerset))
highlights_count: 0
bookmarks_count: 1
booknote_count: 0
2023-05-13
3 :book name: When a Man Marries (免费公版书) (Rinehart,Mary Roberts)
highlights_count: 89
bookmarks_count: 28
booknote_count: 0
2023-05-13
4 :book name: down a narrow road  2009  
highlights_count: 0
bookmarks_count: 3
booknote_count: 0
2023-05-13
5 :book name: The Price of Love (免费公版书) (Bennett,Arnold)
highlights_count: 3
bookmarks_count: 0
booknote_count: 0
2023-05-13
6 :book name: Men are from Mars  
highlights_count: 11
bookmarks_count: 9
booknote_count: 0
2023-05-13
7 :book name: Yulduzli tunlar (roman) (Pirimqul Qodirov)
highlights_count: 114
bookmarks_count: 26
booknote_count: 1
2023-05-13
8 :book name: Avlodlar dovoni (roman) (Pirimqul Qodirov)
h

In [64]:
b ="指数基金投资从入门到精通 (老罗)"

b = "全球风口：积木式创新与中国新机遇 (王煜全;薛兆丰)" 
b ="An Introduction to the History of Science (Walter Libby)"
print(b)

print("------notes---------")
n_l = books[b].notes_list
for note in n_l:
    print(note.content)
    print(note.location)
    print(note.date)
print("------highlights---------")
n_l = books[b].highlights_list
for note in n_l:
    print(note.content)
    print(note.location)
    print(note.date)
print("------bookmarks---------")
n_l = books[b].bookmarks_list
for note in n_l:
#     print(note.content)
    print(note.location)
    print(note.date)
print(books[b].start_finish_reading_date())


An Introduction to the History of Science (Walter Libby)
------notes---------
------highlights---------
------bookmarks---------
263
2020-01-17 09:30:45
bookmarks:
[datetime.datetime(2020, 1, 17, 9, 30, 45)]
(datetime.datetime(2020, 1, 17, 9, 30, 45), datetime.datetime(2020, 1, 17, 9, 30, 45))


In [7]:
from datetime import datetime
#kahar added start
import re
import locale
pattern_1 = re.compile(u' Added on [A-Za-z]+, \d{1,2}')
pattern_2 = re.compile(u' Added on [A-Za-z]+, [A-Za-z]+')
pattern_3 = re.compile(u' 添加于 \d{4}')

#kahar added end

class EditType:
    """
    EditType interface for all types of user generated content on the kindle device
    Some include: Bookmarks, Notes and Highlights. More content could be available but I am not aware of it
    Attributes which it must contain: Date and content
    """

    def parse_edit_location(self, raw_data):
        """
        Parse edit Location or location range
        :param raw_data: String containing location
        :return: Location or location string value
        """

    def parse_edit_date(self, raw_data):
        """
        Parse edit date
        :param raw_data: String containing location
        :return: datetime object
        """


class HighlightType(EditType):
    """
    Created for Kindle Paperwhite gen 5.
    Highlight Edit class.
    Contains content of the highlight, location range of the highlight and the date
    """

    def __init__(self, highlight_string, content):
        self.content = content
        self.date = self.parse_edit_date(highlight_string)
        self.location = self.parse_edit_location(highlight_string)

    def parse_edit_date(self, data):
        """
        Example of input:
        - Your Highlight on Location 6778-6779 | Added on Monday, August 17, 2015 7:38:34 AM
        k:comment: or 17 August so I need to change the code

        :param data:
        :return:
        """
        date_part = data.split("|")
        added_time = ""
        if(pattern_1.search(date_part[-1])):
            locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
            added_time = datetime.strptime(date_part[-1]," Added on %A, %d %B %Y %H:%M:%S")
        elif(pattern_2.search(date_part[-1])):
            locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
            added_time = datetime.strptime(date_part[-1]," Added on %A, %B %d, %Y %I:%M:%S %p")
        elif(pattern_3.search(date_part[-1])):
            locale.setlocale(locale.LC_ALL, 'zh_CN.UTF-8')
            added_time = datetime.strptime(date_part[-1]," 添加于 %Y年%m月%d日%A %p%H:%M:%S")
        else:
            print("{} can't be parsed".format(date_part[-1]))
        return added_time

        #return datetime.strptime(date_part[-1], " Added on %A, %B %d, %Y %I:%M:%S %p")

    def parse_edit_location(self, data):
        """
        Example of input:
        - Your Highlight on Location 6778-6779 | Added on Monday, August 17, 2015 7:38:34 AM

        :param data:
        :return:
        """
        loc_split = data.split("|")
        if len(loc_split) > 2:
            return loc_split[1].rstrip().lstrip().split(" ")[-1]
        else:
            return loc_split[0].split(" ")[5]


class NoteType(EditType):
    """
    Created for Kindle Paperwhite gen 5.
    Highlight Edit class.
    Note Edit class
    Contains content of the Note, location of the note and the date
    """

    def __init__(self, edit_string, content):
        self.content = content
        self.date = self.parse_edit_date(edit_string)
        self.location = self.parse_edit_location(edit_string)

    def parse_edit_date(self, data):
        """
        Example of input:
        - Your Note on Location 4555 | Added on Wednesday, February 24, 2016 8:28:14 AM

        :param data:
        :return:
        """
        date_part = data.split("|")
        #return datetime.strptime(date_part[-1], " Added on %A, %B %d, %Y %I:%M:%S %p")

        added_time = ""
        if(pattern_1.search(date_part[-1])):
            locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
            added_time = datetime.strptime(date_part[-1]," Added on %A, %d %B %Y %H:%M:%S")
        elif(pattern_2.search(date_part[-1])):
            locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
            added_time = datetime.strptime(date_part[-1]," Added on %A, %B %d, %Y %I:%M:%S %p")
        elif(pattern_3.search(date_part[-1])):
            locale.setlocale(locale.LC_ALL, 'zh_CN.UTF-8')
            added_time = datetime.strptime(date_part[-1]," 添加于 %Y年%m月%d日%A %p%H:%M:%S")
        else:
            print("{} can't be parsed".format(date_part[-1]))
        return added_time






    def parse_edit_location(self, data):
        """
        Example of input:
        - Your Note on Location 4555 | Added on Wednesday, February 24, 2016 8:28:14 AM

        :param data:
        :return:
        """
        loc_split = data.split("|")
        if len(loc_split) > 2:
            return loc_split[1].rstrip().lstrip().split(" ")[-1]
        else:
            #kahar add
            if("您" in loc_split[0]):
                print("loc_split:",loc_split[0])
                return loc_split[0].split(" ")[2].strip("#")
            else:
                print("en loc:",loc_split[0])
                return loc_split[0].split(" ")[5]


class BookmarkType(EditType):
    """
    Created for Kindle Paperwhite gen 5.
    Highlight Edit class.
    Bookmark Edit class
    Contains location of the bookmark and the date
    """

    def __init__(self, bookmark_string):
        self.date = self.parse_edit_date(bookmark_string)
        self.location = self.parse_edit_location(bookmark_string)

    def parse_edit_date(self, data):
        """
        Example of input:
        - Your Bookmark on Location 3021 | Added on Wednesday, February 17, 2016 1:49:13 PM

        :param data:
        :return:
        """
        #k: changed
        #k:comment: or 17 August so I need to change the code

        date_part = data.split("|")
        added_time = ""
        if(pattern_1.search(date_part[-1])):
            locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
            added_time = datetime.strptime(date_part[-1]," Added on %A, %d %B %Y %H:%M:%S")
        elif(pattern_2.search(date_part[-1])):
            locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
            added_time = datetime.strptime(date_part[-1]," Added on %A, %B %d, %Y %I:%M:%S %p")
        elif(pattern_3.search(date_part[-1])):
            locale.setlocale(locale.LC_ALL, 'zh_CN.UTF-8')
            added_time = datetime.strptime(date_part[-1]," 添加于 %Y年%m月%d日%A %p%H:%M:%S")
        else:
            print("{} can't be parsed".format(date_part[-1]))
        return added_time



        #date_part = data.split("|")
        #return datetime.strptime(date_part[-1], " Added on %A, %B %d, %Y %I:%M:%S %p")

    def parse_edit_location(self, data):
        """
        Example of input:
        - Your Bookmark on Location 3021 | Added on Wednesday, February 17, 2016 1:49:13 PM

        :param data:
        :return:
        """
        # loc_split = data.split("|")
        # if len(loc_split) > 2:
            # return loc_split[1].rstrip().lstrip().split(" ")[-1]
        # else:
            # return loc_split[0].split(" ")[5]

#kahar added
        loc_split = data.split("|")
        if len(loc_split) > 2:
            return loc_split[1].rstrip().lstrip().split(" ")[-1]
        else:
            #kahar add
            if("您" in loc_split[0]):
                print("loc_split:",loc_split[0])
                return loc_split[0].split(" ")[2].strip("#")
            else:
                print("en loc:",loc_split[0])
                return loc_split[0].split(" ")[5]






In [8]:
books

{'186  ': <__main__.Book at 0x7f8ebc382358>,
 'GrammarAsScience  ': <__main__.Book at 0x7f8ebc385320>,
 'Moon and Sixpence (免费公版书) (Maugham,W. Somerset (William Somerset))': <__main__.Book at 0x7f8ebc1ab5f8>,
 'When a Man Marries (免费公版书) (Rinehart,Mary Roberts)': <__main__.Book at 0x7f8ebc1ab668>,
 'down a narrow road  2009  ': <__main__.Book at 0x7f8ebc1ab978>,
 'The Price of Love (免费公版书) (Bennett,Arnold)': <__main__.Book at 0x7f8ebc224cf8>,
 'Men are from Mars  ': <__main__.Book at 0x7f8ebc224e48>,
 'Yulduzli tunlar (roman) (Pirimqul Qodirov)': <__main__.Book at 0x7f8ebc1c4470>,
 'Avlodlar dovoni (roman) (Pirimqul Qodirov)': <__main__.Book at 0x7f8ebc229278>,
 'Dahshat (hikoyalar) (Abdulla Qahhor)': <__main__.Book at 0x7f8ebc229d30>,
 'Pride and Prejudice (Jane Austen)': <__main__.Book at 0x7f8ebc229eb8>,
 '人类简史：从动物到上帝 (尤瓦尔·赫拉利)': <__main__.Book at 0x7f8ebc1e25c0>,
 '未来简史 ([以色列]尤瓦尔·赫拉利)': <__main__.Book at 0x7f8ebc147940>,
 'Чингизхоннинг оқ булути (қисса) (Chingiz Aytmatov)': <__mai

In [9]:
import locale
locale.ERA_D_T_FMT

131120

In [10]:
locale.getdefaultlocale()

('en_US', 'UTF-8')

In [11]:
locale.setlocale(locale.LC_ALL, 'zh_CN.UTF-8')

'zh_CN.UTF-8'

In [12]:
import re
pattern = re.compile(u' Added on [A-Za-z]+, \d{2}')

str_test ="Added on Wednesday, October 21, 2015 10:45:24 PM"
# str_test =" Added on Monday, 23 November 2015 23:32:48"

data = " 添加于 2018年3月26日星期一 下午7:23:46"
# data = "Added on Wednesday, October 21, 2015 10"
# str_test = u''
match = pattern.search(str_test)
print(match)
if(match):
    print(match.string)



None


In [13]:
import re
pattern = re.compile(u' Added on [A-Za-z]+, \d{2}')
str = u''
print(pattern.search(str))

None
